In [1]:
###################################################################################################
#
# Copyright (C) 2022 Maxim Integrated Products, Inc. All Rights Reserved.
#
# Maxim Integrated Products, Inc. Default Copyright Notice:
# https://www.maximintegrated.com/en/aboutus/legal/copyrights.html
#
###################################################################################################

## IMPORTS
import os
import sys
import datetime
import time

import importlib
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from pathlib import Path
from collections import OrderedDict

import distiller

## FOR TENSOR BOARD
try:
    import tensorboard  # pylint: disable=import-error
    import tensorflow  # pylint: disable=import-error
    tensorflow.io.gfile = tensorboard.compat.tensorflow_stub.io.gfile
except (ModuleNotFoundError, AttributeError):
    pass

import torchnet.meter as tnt
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import distiller.apputils as apputils
from distiller.data_loggers import PythonLogger, TensorBoardLogger

## PATH FOR MODELS AND DATASET
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'models'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'datasets'))

from unetnilm.metrics import get_results_summary, example_f1_score
from datasets import nilm
import ai8x
mod = importlib.import_module("ai87net-unetnilm")

%matplotlib inline

msglogger = None

In [2]:
dataset_fn = nilm.ukdale_get_datasets
data_path = "../data/NILM/"
batch_size = 8
workers = 5
validation_split = 0.1
deterministic = True
dataset_name = "ukdale"
appliance_data = {
    "fridge freezer": {
        "window": 50,
        "min": 0.0,
        "max": 255.0,
        "on_power_threshold": 50
    },
    "washer dryer": {
        "window": 50,
        "min": 0.0,
        "max": 2044.0,
        "on_power_threshold": 20
    },
    "kettle": {
        "window": 50,
        "min": 0.0,
        "max": 2361.0,
        "on_power_threshold": 10
    },
    "dish washer": {
        "window": 10,
        "min": 0.0,
        "max": 2411.0,
        "on_power_threshold": 10
    },
    "microwave": {
        "window": 50,
        "min": 0.0,
        "max": 1547.0,
        "on_power_threshold": 200
    }
}
appliances = list(appliance_data.keys())

## Variable Declaration for MSG Logger
log_prefix = "ukdale-train"
log_dir = "../logs/"
dataset_name = "ukdale"
num_classes = 5
model_name = "cnn1dnilm"
seq_len = 100
lr = 1e-4

## Variable Declaration for Model
quantiles = [0.0025,0.1, 0.5, 0.9, 0.975]

In [3]:
class Args:
        def __init__(self, act_mode_8bit):
                self.act_mode_8bit = act_mode_8bit
                self.truncate_testset = False

def count_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [4]:
args = Args(act_mode_8bit=False)
train_set, test_set, val_set = dataset_fn((data_path, args), load_train=True, load_test=True, load_val=True)
train_set.visualize_batch()

0


In [6]:
train_loader, val_loader, test_loader, _ = apputils.get_data_loaders(
        dataset_fn, (data_path,args), batch_size,
        workers, validation_split, deterministic,1, 1, 1)

In [7]:
batch_size = 128
num_workers = 5

train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)


In [8]:
index =365
data = val_set[index]
input_data = data[0]
target_data = data[1]
state_data = data[2]